In [14]:
from multi_path_base import *
from joblib import Parallel, delayed
%matplotlib tk 
%matplotlib tk 

In [15]:
forecast_with_data=np.load('forecast_with_data.npy')

In [16]:
interpolation_points=10000
x = np.arange(1,72+1,1)

y = forecast_with_data[80,:,1] #X[j,:]
tck = interpolate.splrep(x, y, s=0)

xnew=np.linspace(1,72,interpolation_points)
p_inter = interpolate.splev(xnew, tck, der=0)
p=p_inter

N=len(p_inter)
dt=1
M_test=1
real_1 = real(0.16,0.01)
disct_temp = disct(N,dt,M_test)

In [17]:
L=100
beta_X = np.empty((L,len(p_inter)))
beta_X_derivative_tracking = np.empty((L,len(p_inter)))
beta_V = np.empty((L,len(p_inter)))

normal_X = np.empty((L,len(p_inter)))
normal_X_derivative_tracking = np.empty((L,len(p_inter)))
normal_V = np.empty((L,len(p_inter)))

for j in range(0,L):
    beta_X[j,:]=gen_X_beta(X0=p_inter[0], disct=disct_temp,real=real_1, forecast=p_inter )
    beta_X_derivative_tracking[j,:]=gen_X_beta_derivative_tracking(X0=p_inter[0], disct=disct_temp,real=real_1, forecast=p_inter )
    beta_V[j,:]=gen_path_beta_robust(X0=0, disct=disct_temp,real=real_1, forecast=p_inter )
    
    normal_X[j,:]=gen_X_normal(X0=p_inter[0], disct=disct_temp,real=real_1, forecast=p_inter )
    normal_X_derivative_tracking[j,:]=gen_X_normal_derivative_tracking(X0=p_inter[0], disct=disct_temp,real=real_1, forecast=p_inter )
    normal_V[j,:]=gen_path_normal_robust(X0=0, disct=disct_temp,real=real_1, forecast=p_inter )


-3.225269441919317e-05 6.615309528011404e-10 -6.5184919381021154e-06
-5.2091752346628586e-05 6.47999244631322e-10 -2.6613155557902613e-05
-4.9717448761318374e-05 6.340017250300645e-10 -2.451609642590451e-05
-0.0001117386132529308 6.175779114521605e-10 -8.68375302249407e-05
-0.00014288511597142887 6.009645368835138e-10 -0.00011830580313466295
-0.00017535963567131023 5.834614305018789e-10 -0.0001511241095042785
-0.000243422876969204 5.643260356253213e-10 -0.0002195537024735677
-0.0002449410545141403 5.459556651109754e-10 -0.00022145916290277525
-0.0002612942431088907 5.265291221454075e-10 -0.00023822186802774684
-0.00026778260657950233 5.066679480952827e-10 -0.0002451415868289728
-0.0002765280742583974 4.861506141232489e-10 -0.00025434044294155245
-0.0002974044672981213 4.64810961491871e-10 -0.00027569241585107473
-0.0003096076256436371 4.4318847592009236e-10 -0.0002883930129876943
-0.0003261897069303364 4.2104989655056614e-10 -0.00030549460117321626
-0.0003314766151910159 3.988219726369

In [20]:
fig = plt.figure(100)
fig.clf()

L=100
ax = fig.add_subplot(3,2,1)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,beta_X[j,:], 'k-')
plt.plot(xnew,beta_X[j,:], 'k-', label='simulated forecast')
plt.plot(xnew,p, 'r-', label='forecast')
plt.title('Beta transitions on forecast')
plt.legend()

ax = fig.add_subplot(3,2,4)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,beta_X_derivative_tracking[j,:], 'k-')
plt.plot(xnew,p, 'r-')
plt.title('Beta transitions on forecast with derivative tracking')

ax = fig.add_subplot(3,2,5)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,beta_V[j,:]+p, 'k-')
plt.plot(xnew,p, 'r-')
plt.title('Beta transitions on forecast errors')


ax = fig.add_subplot(3,2,2)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,normal_X[j,:], 'k-')
plt.plot(xnew,p, 'r-')
plt.title('Gaussian transitions on forecast')

ax = fig.add_subplot(3,2,3)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,normal_X_derivative_tracking[j,:], 'k-')
plt.plot(xnew,p, 'r-')
plt.title('Gaussian transitions on forecast with derivative tracking')


ax = fig.add_subplot(3,2,6)
plt.xlim(0, 72)
plt.ylim(0, 1)
for j in range(0,L):
    plt.plot(xnew,normal_V[j,:]+p, 'k-')
plt.plot(xnew,normal_V[j,:]+p, 'k-', label='simulated forecast')
plt.plot(xnew,p, 'r-', label='forecast')
plt.title('Gaussian transitions on forecast errors')

fig.suptitle('Comparison of Models \n $\\theta=0.16, \\alpha=0.01$',fontsize=24)

plt.savefig('model_comparison.pdf')
#plt.legend()